#Boosted Random Forest Regression

##Data preprocessing

In [0]:
import numpy as np
import pandas as pd
import datetime as dt
import sklearn
from scipy import stats
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

In [0]:
data = pd.read_csv('All India Air Quality Data.csv',encoding='cp1252')                                                                                                                         

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
print(data.columns)

Index(['stn_code', 'sampling_date', 'state', 'location', 'agency', 'type',
       'so2', 'no2', 'rspm', 'spm', 'location_monitoring_station', 'pm2_5',
       'date'],
      dtype='object')


In [0]:
print(data.head)

In [0]:
encoder = preprocessing.LabelEncoder()
regressor = AdaBoostRegressor(RandomForestRegressor(),loss='exponential',learning_rate=0.01)

data['date'] = pd.to_datetime(data['date'])
data['date']= data['date'].map(dt.datetime.toordinal)

In [0]:
print(data.columns)

Index(['stn_code', 'sampling_date', 'state', 'location', 'agency', 'type',
       'so2', 'no2', 'rspm', 'spm', 'location_monitoring_station', 'pm2_5',
       'date'],
      dtype='object')


In [0]:
data = data.fillna(np.nan,axis=0)


data['stn_code'] = encoder.fit_transform(data['stn_code'].astype(str))
data['state'] = encoder.fit_transform(data['state'])
data['location'] = encoder.fit_transform(data['location'].astype(str))
data['type'] = encoder.fit_transform(data['type'].astype(str))
data['location_monitoring_station'] = encoder.fit_transform(data['location_monitoring_station'].astype(str))

In [0]:
print(data['date'].head())

0    726499
1    726499
2    726499
3    726527
4    726527
Name: date, dtype: int64


##Training Set: 80%  ; Test Set: 20%

###Sulphur Dioxide

In [0]:
dat1 = data[['stn_code','state','location','type','so2','location_monitoring_station','date']]

In [0]:
dat1['so2'] = dat1['so2'].fillna(np.mean(dat1['so2']),axis=0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:

z = np.abs(stats.zscore(dat1))   #Detecting outliers
print(z.shape)

dat1 = dat1[(z<3).all(axis=1)]
print(z.shape)

(435742, 7)
(435742, 7)


In [0]:

X1 = dat1[['stn_code','state','location','type','location_monitoring_station','date']]
Y1 = dat1['so2']

In [0]:

X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X1,Y1,test_size=0.2,random_state=0)

regressor.fit(X_train1,np.array(Y_train1).reshape(Y_train1.shape[0],1))


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/fo

KeyboardInterrupt: ignored

In [0]:
pred1 = np.array(regressor.predict(X_test1))

cod1 = regressor.score(X_test1,np.array(Y_test1).reshape(Y_test1.shape[0],1))
ms1 = mse(pred1,np.array(Y_test1).reshape(Y_test1.shape[0],1))
ma1 = mae(pred1,np.array(Y_test1).reshape(Y_test1.shape[0],1))

In [0]:
print('*** Evaluation metrics for SO2 ***\n')
print('Coeff. of Determination: ',cod1)
print('Mean Squared Error: ',ms1)
print('Root Mean Sqaured Error: ',ms1**0.5)
print('Mean Absolute Error: ',ma1)
a = pd.DataFrame(Y_test1)
a['pred']= regressor.predict(X_test1)
print('\n\tTable 1\n')
print(a.head())

###Nitrogen Dioxide

In [0]:
dat2 = data[['stn_code','state','location','type','no2','location_monitoring_station','date']]

In [0]:
dat2['no2'] = dat2['no2'].fillna(np.mean(dat2['no2']),axis=0)

In [0]:
z = np.abs(stats.zscore(dat2))   #Detecting outliers

dat2 = dat2[(z<3).all(axis=1)]

In [0]:
X2 = dat2[['stn_code','state','location','type','location_monitoring_station','date']]
Y2 = dat2['no2']

In [0]:
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X2,Y2,test_size=0.2,random_state=0)
regressor.fit(X_train2,np.array(Y_train2).reshape(Y_train2.shape[0],1))

In [0]:
pred2 = np.array(regressor.predict(X_test2))

cod2 = regressor.score(X_test2,np.array(Y_test2).reshape(Y_test2.shape[0],1))
ms2 = mse(pred2,np.array(Y_test2).reshape(Y_test2.shape[0],1))
ma2 = mae(pred2,np.array(Y_test2).reshape(Y_test2.shape[0],1))

In [0]:
print('*** Evaluation metrics for NO2 ***\n')
print('Coeff. of Determination: ',cod2)
print('Mean Squared Error: ',ms2)
print('Root Mean Sqaured Error: ',ms2**0.5)
print('Mean Absolute Error: ',ma2)
a = pd.DataFrame(Y_test2)
a['pred']= regressor.predict(X_test2)
print('\n\tTable 1\n')
print(a.head())

###Respirable Suspended Particulate Matter

In [0]:
dat3 = data[['stn_code','state','location','type','rspm','location_monitoring_station','date']]

In [0]:
dat3['rspm'] = dat3['rspm'].fillna(np.mean(dat3['rspm']),axis=0)

In [0]:
z = np.abs(stats.zscore(dat3))   #Detecting outliers
print(z.shape)

dat3 = dat3[(z<3).all(axis=1)]
print(z.shape)

In [0]:
X3 = dat3[['stn_code','state','location','type','location_monitoring_station','date']]
Y3 = dat3['rspm']

In [0]:
X_train3, X_test3, Y_train3, Y_test3 = train_test_split(X3,Y3,test_size=0.2,random_state=0)
regressor.fit(X_train3,np.array(Y_train3).reshape(Y_train3.shape[0],1))

In [0]:
pred3 = np.array(regressor.predict(X_test3))

cod3 = regressor.score(X_test3,np.array(Y_test3).reshape(Y_test3.shape[0],1))
ms3 = mse(pred3,np.array(Y_test3).reshape(Y_test3.shape[0],1))
ma3 = mae(pred3,np.array(Y_test3).reshape(Y_test3.shape[0],1))

In [0]:
print('*** Evaluation metrics for RSPM ***\n')
print('Coeff. of Determination: ',cod3)
print('Mean Squared Error: ',ms3)
print('Root Mean Sqaured Error: ',ms3**0.5)
print('Mean Absolute Error: ',ma3)
a = pd.DataFrame(Y_test3)
a['pred']= regressor.predict(X_test3)
print('\n\tTable 3\n')
print(a.head())

###Suspended Particulate Matter

In [0]:
dat4 = data[['stn_code','state','location','type','spm','location_monitoring_station','date']]

In [0]:
dat4['spm'] = dat4['spm'].fillna(np.mean(dat4['spm']),axis=0)

In [0]:
z = np.abs(stats.zscore(dat4))   #Detecting outliers

dat4 = dat4[(z<3).all(axis=1)]

In [0]:
X4 = dat4[['stn_code','state','location','type','location_monitoring_station','date']]
Y4 = dat4['spm']

In [0]:
X_train4, X_test4, Y_train4, Y_test4 = train_test_split(X4,Y4,test_size=0.2,random_state=0)
regressor.fit(X_train4,np.array(Y_train4).reshape(Y_train4.shape[0],1))

In [0]:
pred4 = np.array(regressor.predict(X_test4))

cod4 = regressor.score(X_test4,np.array(Y_test4).reshape(Y_test4.shape[0],1))
ms4 = mse(pred4,np.array(Y_test4).reshape(Y_test4.shape[0],1))
ma4 = mae(pred4,np.array(Y_test4).reshape(Y_test4.shape[0],1))

In [0]:
print('*** Evaluation metrics for SPM ***\n')
print('Coeff. of Determination: ',cod4)
print('Mean Squared Error: ',ms4)
print('Root Mean Sqaured Error: ',ms4**0.5)
print('Mean Absolute Error: ',ma4)
a = pd.DataFrame(Y_test4)
a['pred']= regressor.predict(X_test4)
print('\n\tTable 4\n')
print(a.head())

###Particulate Matter 2.5 (PM2.5)

In [0]:
dat5 = data[['stn_code','state','location','type','pm2_5','location_monitoring_station','date']]

In [0]:
dat5['pm2_5'] = dat5['pm2_5'].fillna(np.mean(dat5['pm2_5']),axis=0)

In [0]:
z = np.abs(stats.zscore(dat5))   #Detecting outliers
dat5 = dat5[(z<3).all(axis=1)]

In [0]:
X5 = dat5[['stn_code','state','location','type','location_monitoring_station','date']]
Y5 = dat5['pm2_5']

In [0]:
X_train5, X_test5, Y_train5, Y_test5 = train_test_split(X5,Y5,test_size=0.2,random_state=0)
regressor.fit(X_train5,np.array(Y_train5).reshape(Y_train5.shape[0],1))

In [0]:
pred5 = np.array(regressor.predict(X_test5))

cod5 = regressor.score(X_test5,np.array(Y_test5).reshape(Y_test5.shape[0],1))
ms5 = mse(pred5,np.array(Y_test5).reshape(Y_test5.shape[0],1))
ma5 = mae(pred5,np.array(Y_test5).reshape(Y_test5.shape[0],1))

In [0]:
print('*** Evaluation metrics for PM2.5 ***\n')
print('Coeff. of Determination: ',cod5)
print('Mean Squared Error: ',ms5)
print('Root Mean Sqaured Error: ',ms5**0.5)
print('Mean Absolute Error: ',ma5)
a = pd.DataFrame(Y_test5)
a['pred']= regressor.predict(X_test5)
print('\n\tTable 5\n')
print(a.head())

##Visualization

In [0]:
import matplotlib.pyplot as plt

In [0]:
coef_of_deter = [cod1,cod2,cod3,cod4,cod5]
msqe = [ms1,ms2,ms3,ms4,ms5]
mabs = [ma1,ma2,ma3,ma4,ma5]



In [0]:

def plotter(ht,title):
  label = ['so2','no2','rspm','spm','pm2_5']
  quant = ht
  index = np.arange(len(label))
  colors= ['r','g','b','m','c']
  plt.title(title)
  plt.plot(quant,color=np.random.choice(colors),marker='*')
  plt.bar(index,quant,color=np.random.choice(colors))
  
  plt.xlabel('Components', fontsize=10)
  plt.ylabel('Results', fontsize=10)
  plt.xticks(index, label, fontsize=10)
  plt.show()

###Coefficient of Determination

In [0]:
plotter(ht=coef_of_deter,title='Coefficient of Determination')

###Mean Absolute Error

In [0]:
plotter(ht=mabs,title='Mean Absolute Error')

###Mean Squared Error

In [0]:
plotter(ht=msqe,title='Mean Squared Error')